In [9]:
import sys
import os

sys.path.append("ECCOv4-py")
import ecco_v4_py as ecco

from ecco_download import *

In [11]:
download_dir = os.getcwd()
print(download_dir)

/Users/emiliolr/Desktop/GTC-OTP


In [12]:
ecco_podaac_download(ShortName = "ECCO_L4_SSH_LLC0090GRID_DAILY_V4R4",
                     StartDate = "2000-01-08", 
                     EndDate = "2000-01-14", 
                     download_root_dir = download_dir,
                     n_workers = 6, 
                     force_redownload = False)

created download directory /Users/emiliolr/Desktop/GTC-OTP/ECCO_L4_SSH_LLC0090GRID_DAILY_V4R4

Total number of matching granules: 7
DL Progress: 100%|###########################| 7/7 [00:09<00:00,  1.42s/it]

total downloaded: 41.48 Mb
avg download speed: 4.17 Mb/s
Time spent = 9.954829931259155 seconds
